In [1]:
import os, geopandas, pandas as pd
import country_converter as coco
from pyproj import CRS

In [2]:
current_folder = globals()['_dh'][0]
rootdir = os.path.dirname(os.path.dirname(current_folder))
indir_inter = os.path.join(rootdir, '_2_intermediate', 'data')
indir_prepro = os.path.join(rootdir, '_1_preprocessing', 'data', 'raw', 'gis')
outdir = os.path.join(rootdir, '_3_figures_tables', 'data')

In [3]:
wcea_crs = CRS.from_string('esri:54034') # World cylindical equal area projection

In [4]:
admin_gdf = geopandas.read_file(os.path.join(indir_prepro, 'districts_for_plotting', 'districts_for_plotting.shp'))[['iso', 'district', 'geometry']]
df = pd.read_csv(os.path.join(indir_inter,'pop_n_land_size_of_districts.csv'))[['iso', 'census_year', 'ndist', 'mean_pop', 'med_pop', 'std_pop']]
df.columns = ['iso', 'census_year', 'n_districts', 'pop_mean', 'pop_median', 'pop_std']

In [5]:
admin_gdf = admin_gdf.to_crs(crs=wcea_crs)
admin_gdf["area"] = admin_gdf['geometry'].area/ 10**6
admin_gdf = admin_gdf.groupby('iso').agg({'area': ['mean', 'median', 'std']}).reset_index(drop=False)
admin_gdf.columns = ['iso', 'area_mean', 'area_median', 'area_std']

In [6]:
df = pd.merge(
    df,
    admin_gdf,
    on=['iso'],
    how='left'
)

In [7]:
df

,iso,census_year,n_districts,pop_mean,pop_median,pop_std,area_mean,area_median,area_std
0,BEN,2013.0,77,1.311290e+05,110540.000,1.029679e+05,1497.341710,528.319229,1707.522713
1,BWA,2011.0,21,9.607238e+04,71530.000,7.286199e+04,27531.438669,23913.644799,29285.130035
2,BFA,2006.0,45,2.790822e+05,222890.000,2.676985e+05,6064.748161,5364.413998,3245.884137
3,CMR,2005.0,204,8.430583e+04,48360.000,1.922566e+05,1906.235932,744.698467,2791.236154
4,EGY,2006.0,235,3.098908e+05,264160.000,2.905518e+05,4157.363213,154.189578,29683.777276
5,ETH,2007.0,89,8.157597e+05,462853.490,7.875424e+05,11759.397941,8182.715208,13744.573521
6,GHA,2010.0,110,2.242081e+05,166770.000,2.775268e+05,2173.570545,1383.674865,2404.326491
7,GIN,2014.0,33,3.184594e+05,278290.000,2.756073e+05,7421.992733,6010.842419,4564.305062
8,LBR,2008.0,42,8.205000e+04,44680.000,1.515064e+05,2283.763938,1525.105409,2337.569751
9,MWI,2008.0,183,7.333208e+04,55280.000,7.833849e+04,632.122233,343.946965,1705.335502


**Note**
- no shapefile for Mauritius
- one district drops for Ethiopia since it is not included in the most recent (2007) census